In [12]:
# Some jupyter notebook magic to reload modules automaticaally when they change
# not neccessary for this specific notebook but useful in general
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
from definitions import ROOT_DIR
import os

# The PSDM specific input models can be imported from the pypsdm.models.input and
# pypsdm.models.result
from pypsdm.models.input import GridContainer


grid_path = os.path.join(ROOT_DIR, "tests", "resources", "simple_grid", "input")
# IO data models in general have a from_csv method to parse psdm files
grid = GridContainer.from_csv(grid_path)

In [14]:
from pypsdm.plots.grid import grid_plot

# Use the grid_plot method to visualize the grid model
# only works if the underlying node input files have associated coordinates
grid_plot(grid)

In [15]:
# A grid container consists of a raw grid container
raw_grid = grid.raw_grid
# consisting of lines, transformers and so on
lines = raw_grid.lines
# and a participants container
participants = grid.participants
# consisting of loads, pv plants
pvs = participants.pvs

# You can also directly access the nested objects from the container classes
pvs = grid.pvs

# The base data structure of all input model is a pandas DataFrame accessible via .data
pvs.data

,id,operates_from,operates_until,operator,node,q_characteristics,s_rated,albedo,azimuth,elevation_angle,eta_conv,k_g,k_t,market_reaction,cos_phi_rated
uuid,,,,,,,,,,,,,,,
35b25f11-434b-49c8-acba-797a986f52b4,PV_NS_02,NaN,NaN,NaN,b7a5be0d-2662-41b2-99c6-3b8121a75e9e,"cosPhiFixed:{(0.0, 0.9)}",4,0.21,90,40,97,0.9,1,False,0.9
d4f0ff3a-a44d-4ec9-9f23-a435acd8fbc9,PV_NS_03,NaN,NaN,NaN,1dcddd06-f41a-405b-9686-7f7942852196,"cosPhiFixed:{(0.0, 0.9)}",6,0.21,0,45,97,0.9,1,False,0.9
718ac721-c577-4341-b1d0-f7545d56fa2d,PV_NS_04,NaN,NaN,NaN,e3c3c6a3-c383-4dbb-9b3f-a14125615386,"cosPhiFixed:{(0.0, 0.9)}",10,0.21,-90,30,97,0.9,1,False,0.9


In [16]:
# You can access the columns via the data frame
pvs.data["s_rated"]

uuid
35b25f11-434b-49c8-acba-797a986f52b4     4
d4f0ff3a-a44d-4ec9-9f23-a435acd8fbc9     6
718ac721-c577-4341-b1d0-f7545d56fa2d    10
Name: s_rated, dtype: int64

In [17]:
# or directly via the property attribute of the class
pvs.s_rated

uuid
35b25f11-434b-49c8-acba-797a986f52b4     4
d4f0ff3a-a44d-4ec9-9f23-a435acd8fbc9     6
718ac721-c577-4341-b1d0-f7545d56fa2d    10
Name: s_rated, dtype: int64

In [18]:
# The respective classes implement some useful methods for dealing with the data
# e.g. retrieve all elements connected to one or multiple specified nodes
# (Please check out the classes to see all the implemented methods)
nodal_participants = participants.filter_by_nodes(grid.nodes.uuid.to_list()[::2])
nodal_participants.pvs.data

,id,operates_from,operates_until,operator,node,q_characteristics,s_rated,albedo,azimuth,elevation_angle,eta_conv,k_g,k_t,market_reaction,cos_phi_rated
uuid,,,,,,,,,,,,,,,
35b25f11-434b-49c8-acba-797a986f52b4,PV_NS_02,NaN,NaN,NaN,b7a5be0d-2662-41b2-99c6-3b8121a75e9e,"cosPhiFixed:{(0.0, 0.9)}",4,0.21,90,40,97,0.9,1,False,0.9
718ac721-c577-4341-b1d0-f7545d56fa2d,PV_NS_04,NaN,NaN,NaN,e3c3c6a3-c383-4dbb-9b3f-a14125615386,"cosPhiFixed:{(0.0, 0.9)}",10,0.21,-90,30,97,0.9,1,False,0.9


In [20]:
from tempfile import TemporaryDirectory, tempdir

# You can write data models with their respective to_csv method
with TemporaryDirectory() as temp_dir:
    grid.to_csv(temp_dir)
    grid_from_csv = GridContainer.from_csv(temp_dir)

    grid.compare(grid_from_csv)
    assert grid.raw_grid.nodes == grid_from_csv.raw_grid.nodes
    assert grid.loads == grid_from_csv.loads
    # the == operator is overloaded to compare the underlying dataframes
    assert grid == grid_from_csv

True
